In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json

In [ ]:
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

In [ ]:
unzip data.zip

In [ ]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, GlobalAveragePooling2D, Rescaling
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
model = Sequential([
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    # Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    
    # Dropout(0.5),
    Dense(units=1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.002, momentum=0.8),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    training_set,
    validation_data= validation_set,
    epochs=10,
    # callbacks=[reduce_lr, early_stop]
)

In [ ]:
history.history

In [ ]:
model.save('trainedv3.h5')

In [ ]:
train_loss, train_acc = model.evaluate(training_set)

In [ ]:
print(train_loss, train_acc)

In [ ]:
with open ('training_histv3.json', 'w') as f:
    json.dump(history.history,f)

In [ ]:
epochs = [i for i in range(1,11)]
plt.plot(epochs, history.history['accuracy'], color='red', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], color='blue', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
# plt.savefig("output.jpg")
# Saving figure by changing parameter values
# plt.savefig("output1", facecolor='y', bbox_inches="tight",
            pad_inches=0.3, transparent=True)
plt.show()

In [ ]:
epochs = [i for i in range(1,6)]
plt.plot(epochs, history.history['loss'], color='red', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], color='blue', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
# plt.savefig("outputloss2.jpg")
plt.show()